# Objetivo del notebook

El objetivo de este notebook es desarrollar todas las funciones que creamos en nuestro archivo funciones.py y explicar que conseguimos con ellas

<br>
<br>

### Función transformación de "df_electricity"

La funcion es:

In [4]:
def electricity_transformation(df):
    
    # Crear nuevas columnas 'hour' y 'day'
    df['datetime'] = pd.to_datetime(df['forecast_date'])
    df['hour'] = df['datetime'].dt.hour
    df['date'] = df['datetime'].dt.date

    # Calcular la media por día y asignarla a la columna 'price_per_day'
    df["price_per_day"] = df.groupby('date')['euros_per_mwh'].transform("mean")

    # Calcular la diferencia de precio con respecto al valor anterior
    df["price_diff_with_previous"] = df["euros_per_mwh"].diff()

    # Crear columnas para los precios anteriores con shift
    df['previous_price_t-hour'] = df['euros_per_mwh'].shift(1) # 1 hora
    df['previous_price_t-day'] = df['euros_per_mwh'].shift(24) # 1 dia
    df['previous_price_t-week'] = df['euros_per_mwh'].shift(168) # 1 semana
    df['previous_price_t-month'] = df['euros_per_mwh'].shift(720) #  1 mes
    
    # Eliminamos la columna
    df.drop(columns=['forecast_date'], inplace=True)

    return df

- Creamos la variable "datetime" a partir de la variable "forecast_date" con formato fecha-hora" lo cual nos va ayudar a unir con la tabla df_train en un futuro
- Extraemos la hora y la fecha por separado en dos columnas diferentes
- Calculamos la media del precio de la electricidad para cada dia y lo agregamos a una nueva columna
- Creamos una nueva columna en la que calculamos la diferencia de precio con la hora anterior, esto nos sirve para ver las tendencias de precio
- Creamos nuevas columnas con el precio de la hora anterior, día, semana y mes
- Por ultimo eliminamos la colum

<br>
<br>

### Función transformación de "df_gas"

La función es:

In [5]:
def gas_transformation(df):
    
    # Pasamos a formato fecha para poder unir
    df['date'] = pd.to_datetime(df['forecast_date'])
    
    # Creamos la columna average price 
    df['average_price'] = df[['lowest_price_per_mwh' , 'highest_price_per_mwh']].mean(axis=1)
    
    # Creamos la columna price_difference
    df['price_difference'] = df['highest_price_per_mwh'] - df['lowest_price_per_mwh']
    
    # Eliminamos la columna
    df.drop(columns=['forecast_date'], inplace=True)
    
    return df

- En este caso "forecast_date" solo nos indica la fecha y no la hora, por lo que la usamos para crear la variable "date"
- Calculamos la media del precio para cada día y lo agregamos a una nueva columna
- También creamos una columna con la diferencia entre el precio máximo y mínimo
- Finalmente eliminamos la columna "forecast_date"

<br>
<br>

### Función transformacion de "df_client"

La función es:

In [6]:
def client_transformation(df):
    
    # Convertimos a formato fecha
    df['date'] = pd.to_datetime(df['date'])
    
    # Proporción de la capacidad instalada con respecto al total
    df['capacity_ratio'] = df['installed_capacity'] / df.groupby('date')['installed_capacity'].transform('sum')
    
    return df

- Convertimos a formato fecha la columna "date"
- Calculamos la capacidad instalada con respecto al total, agrupando en este caso por día

<br>
<br>

### Función transformacion de "df_train"

La función es:

In [7]:
def train_transformation(df, holiday):
    
    # Formato fecha-hora
    df['datetime'] = pd.to_datetime(df['datetime'])
    
    # Crear nuevas columnas derivadas
    df['date'] = df['datetime'].dt.date
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['datetime'].dt.year # año
    df['month'] = df['datetime'].dt.month # mes
    df['hour'] = df['datetime'].dt.hour # hora
    df['day_of_month'] = df['datetime'].dt.day # dia del mes
    df['day_of_week'] = df['datetime'].dt.day_of_week # dia de la semana
    
    # Creamos la columna que indica si es festivo o no
    df['holiday'] =  df['date'].isin(holiday).astype(int)
    
    # Lagged-target
    df['lagged_target_1day'] = df.groupby(['prediction_unit_id', 'is_consumption'])['target'].shift(24) # 1 dia
    df['lagged_target_2day'] = df.groupby(['prediction_unit_id', 'is_consumption'])['target'].shift(48) # 2 dias
    df['lagged_target_3day'] = df.groupby(['prediction_unit_id', 'is_consumption'])['target'].shift(72) # 3 dias
    df['lagged_target_4day'] = df.groupby(['prediction_unit_id', 'is_consumption'])['target'].shift(96) # 4 dias
    df['lagged_target_5day'] = df.groupby(['prediction_unit_id', 'is_consumption'])['target'].shift(120) # 5 dias
    df['lagged_target_6day'] = df.groupby(['prediction_unit_id', 'is_consumption'])['target'].shift(144) # 6 dias
    df['lagged_target_7day'] = df.groupby(['prediction_unit_id', 'is_consumption'])['target'].shift(168) # 7 dias
    df['lagged_target_15day'] = df.groupby(['prediction_unit_id', 'is_consumption'])['target'].shift(360) # 15 dias
    df['lagged_target_1month'] = df.groupby(['prediction_unit_id', 'is_consumption'])['target'].shift(720) # 30 dias
    
    # Tendencia
    df['target_trend'] = df['lagged_target_2day'] - df['lagged_target_1day']
    df['target_ratio'] = np.where(df['lagged_target_1day'] != 0, (df['lagged_target_2day'] - df['lagged_target_1day']) / df['lagged_target_1day'], np.nan)
    df['target_diff_seasonal'] = df['lagged_target_7day'] - df['lagged_target_1day']
    
    # Columnas que indican si es findesemana o no
    df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(bool)
    df['is_working_day'] = ~df['is_weekend'].astype(bool)
    
    # Agrega columnas de funciones seno y coseno para la fecha y hora
    df['sin_datetime'] = np.sin(2 * np.pi * df['datetime'].dt.dayofyear / 365)
    df['cos_datetime'] = np.cos(2 * np.pi * df['datetime'].dt.dayofyear / 365)
    
    # Eliminamos esta columna ya que no nos aporta valor
    df.drop(columns=['day_of_week'], inplace=True)

    return df

- Convertimos a formato fecha-hora la columna "datetime"
- A partir de dicha columna sacamos las siguientes:
    - Fecha (la cual pasamos a formato fecha)
    - Año
    - Mes
    - Hora
    - Día del mes
    - Día de la semana
- Creamos una columna booleana que indica si es festivo o no (para ello necesita un listado con la fecha de los dias festivos, lo cual obtendremos mediante web-scrapping con otra función)

Consideramos que la producción de las dias anteriores sería bastante util para predecir los días siguientes, pero teniamos dos inconvenientes:
- Hay tipos de clientes con características distintas
- La variable "target" refleja tanto consumo como producción, dependiendo de la variable "is_consumption"

Por ello agrupamos por "predict_unit_id", ya que cada valor de dicha columna agrupa a clientes con las mismas características y también agrupamos por "is_consumption" para que si se trataba de producción buscara un dato anterior también de producción

Con esto conseguimos crear las siguientes columnas:
- target de las siguientes fechas anteriores:
    - 1 día
    - 2 días
    - 3 días
    - 4 días
    - 5 días
    - 6 días
    - 7 días
    - 15 días
    - 30 días

Usando dichas columnas creamos las siguientes:
- Tendencia, restando los dos dias anteriores
- Ratio de cambio entre las columna "lagged_target_2day" y lagged_target_1day" siempre y cuando este ultimo no sea 0, en ese caso se añadirá un valor nulo
- Diferencia entre el target de hace 7 días y el de hace 1 día

A partir de la columna "day_of_week" creamos dos columnas:
- Fin de semana
- Día de trabajo

Preveiamos que habría cierta estacionalidad en los datos, por lo que creamos las dos siguientes columnas:
- Seno de la fecha
- Coseno de la fecha

Ambas representan la variación cíclica a lo largo del tiempo

Finalmente eliminamos la columna "day_of_week" la cual solo creamos para poder obtener los dias de trabajo y fin de semana

<br>
<br>

### Función transformación de "df_historical"

La función es:

In [8]:
def historical_transformation(df, df_location):
    
    # Guardamos estas columnas para usar despues
    dates = df['datetime']
    latitude = df['latitude']
    longitude = df['longitude'] 
    
    # Borramos las columnas que no aportan valor
    columas_elim = ['latitude', 'longitude', 'data_block_id', 'datetime','rain', 'snowfall', 'winddirection_10m', 'windspeed_10m', 'cloudcover_high', 'cloudcover_mid']
    df = df.drop(columns = columas_elim, axis=1)
    
    # Scaler
    scaler = StandardScaler().fit(df)
    dt = scaler.transform(df)
    df_historical_scaled = pd.DataFrame(dt, columns=df.columns)
    
    # Aplicamos pca de 4, despues de estudiar cual es el mejor numero de componentes
    pca = PCA(n_components=4, random_state = 42) 
    pca = pca.fit(df_historical_scaled)
    df_historical_transformed = pca.transform(df_historical_scaled)
    
    # Utilizamos este número de clusters porque son los más adecuados, despues de realizar un estudio
    kmeans = KMeans(n_clusters=4, n_init = "auto")
    kmeans_labels = kmeans.fit(df_historical_transformed)
    kmeans.fit(df_historical_transformed)
    labels = kmeans.predict(df_historical_transformed)

    # Agregar los clústers al DataFrame original
    df['labels'] = labels
    
    # Utilizamos este número de clusters porque son los más adecuados, despues de realizar un estudio, para lograr un Kmeans mas especifico
    kmeans = KMeans(n_clusters=10, n_init = 'auto')
    kmeans_labels = kmeans.fit(df_historical_transformed)
    kmeans.fit(df_historical_transformed)
    specific_labels = kmeans.predict(df_historical_transformed)

    # Agregar los clústers al DataFrame original
    df['specific_labels'] = specific_labels

    # Agregamos la columna datetime al DataFrame original
    df['datetime'] = dates
    df['latitude'] = latitude
    df['longitude'] = longitude
    
    # Unimos con la tabla df_location para agregar la columna "county"
    df = pd.merge(df, df_location, how='left', on=['longitude', 'latitude']) 
    
    # Creamos una columna que diferencie temperaturas
    df['temperature_dewpoint_diff_hist'] = df['temperature'] - df['dewpoint']
    
    # Radiación solar total ajustada por la cobertura de nubes
    df['adjusted_solar_radiation'] = df['shortwave_radiation'] * (1 - df['cloudcover_total'])
    
    # Relación entre la temperatura y la presión atmosférica
    df['temperature_pressure_ratio'] = df['temperature'] / df['surface_pressure']
    
    # Sacamos la hora, el dia y el mes para poder obtener variables segun la hora, el dia y el mes para cada region
    df['datetime'] = pd.to_datetime(df['datetime'])
    df['hour'] = df['datetime'].dt.hour
    df['day'] = df['datetime'].dt.day_of_year
    df['month'] = df['datetime'].dt.month
    
    # Creamos una columna de temperatura media por hora, dia y mes para cada region
    df['temperature_per_hour_hist'] = df.groupby(['hour', 'county'])['temperature'].transform(lambda x: x.expanding().mean())
    df['temperature_per_day_hist'] = df.groupby(['day', 'county'])['temperature'].transform(lambda x: x.expanding().mean())
    df['temperature_per_month_hist'] = df.groupby(['month', 'county'])['temperature'].transform(lambda x: x.expanding().mean())
    
    # Creamos una columna con la nubosidad, por hora, dia y mes para cada region
    df['cloudcover_total_per_hour'] = df.groupby(['hour', 'county'])['cloudcover_total'].transform(lambda x: x.expanding().mean())
    df['cloudcover_total_per_day'] = df.groupby(['day', 'county'])['cloudcover_total'].transform(lambda x: x.expanding().mean())
    df['cloudcover_total_per_month'] = df.groupby(['month', 'county'])['cloudcover_total'].transform(lambda x: x.expanding().mean())
    
    # Creamos una columna con la shortwave, por hora, dia y mes para cada region
    df['shortwave_rad_per_hour'] = df.groupby(['hour', 'county'])['shortwave_radiation'].transform(lambda x: x.expanding().mean())
    df['shortwave_rad_per_day'] = df.groupby(['day', 'county'])['shortwave_radiation'].transform(lambda x: x.expanding().mean())
    df['shortwave_rad_per_month'] = df.groupby(['month', 'county'])['shortwave_radiation'].transform(lambda x: x.expanding().mean())
    
    # Creamos una columna con la radiacion directa, por hora, dia y mes para cada region
    df['direct_solar_rad_per_hour_hist'] = df.groupby(['hour', 'county'])['direct_solar_radiation'].transform(lambda x: x.expanding().mean())
    df['direct_solar_rad_per_day_hist'] = df.groupby(['day', 'county'])['direct_solar_radiation'].transform(lambda x: x.expanding().mean())
    df['direct_solar_rad_per_month_hist'] = df.groupby(['month', 'county'])['direct_solar_radiation'].transform(lambda x: x.expanding().mean())
    
    # Creamos una lista con las columnas a las que hacerle la media
    col_mean = [col for col in df.columns if col not in ['labels', 'specific_labels', 'datetime', 'county']]
    
    # Hacemos la media de las columnas por county y en la columna de "labels" utilizamos la moda
    df = df.groupby(['datetime', 'county']).agg({'labels': lambda x: x.mode().iloc[0], 
                                                 'specific_labels': lambda x: x.mode().iloc[0],
                                                 **{col: 'mean' for col in col_mean}}).reset_index()
    
    return df

Al igual que en la funcion de "train_transformation" queriamos acceder a los valores de "target" de fechas anteriores de clientes similares, queriamos hacer lo mismo, pero además añadiendole que en ese momento existieran unas condiciones climatológicas similares, para ello tuvimos que hacer lo siguiente:
- Teniamos que hacer agrupaciones de las variables climatológicas, y decidimos usar KMeans para lograrlo, por lo que necesitamos realizar lo siguiente:
    - Guardar las columnas: "datetime", "latitude" y "longitude" ya que ahora las teniamos que eliminar pero posteriormente nos haría falta
    - Eliminamos las columnas que no nos aportaban valor al KMeans
    - Realizamos PCA para reducir la dimensionalidad (anteriormente hicimos un estudio para ver el mejor numero de componentes)

Antes de ejecutar KMeans vimos cuales eran las mejores opciones para el número de clusters, la mejor opción era 2 (la cual descartamos ya que simplemente diferenciaría entre día y noche y queriamos algo más específico), seguido de 10 y 4

El problema era que si poníamos 10 se volvia tan especifico que cuando queriamos laggear varias veces generaba muchos nulos, ya que le costaba mucho encontrar datos anteriores

Finalmente decidimos crear 2 columnas:
- Cluster de 4, sobre la que laggearemos varias veces
- Cluster de 10, sobre el que solo laggearemos una vez

- Volvemos a agregar las columnas que habiamos guardado anteriormente
- Unimos con la tabla "df_location" para obtener la columna "county"
- Creamos una variable sobre la diferencia entre "temperature" y "dew_point", ya que son dos variables relacionadas con la temperatura
- Radiación en relación a la nubosidad
- Relación temperatura y presión atmosférica
- Convertir a formato fecha-hora la columna "datetime" y sacamos de ahí las siguientes columnas:
    - Hora
    - Día
    - Mes

Ahora queriamos crear nuevas columnas con la media por hora, dia y mes, pero solo con los datos hasta ese momento, no los futuros, por lo que tuvimos que hacer medias rodantes de las siguientes variables:
- Temperatura
- Nubosidad total
- Radiación de ondas cortas
- Radiación total

Finalmente tuvimos otro problema, al estar los datos por longitud y latitud, teniamos diferentes filas para una misma fecha-hora en un mismo county, y solo podiamos tener una fila para cada fecha-hora y county, ya que de esa manera estaban estructurados los datos en "df_train" donde teniamos nuestra variable objetivo

Entonces teniamos que como transformar esos datos y para ello hicimos lo siguiente:
- Agrupamos por fecha-hora y county
- Hicimos la moda para "labels" y "specific_labels", ya que estas columnas realmente eran de tipo objeto y las necesitabamos para laggear más adelante
- Hicimos la media para el resto de variables

<br>
<br>

### Función para obtener los días festivos de Estonia

La función es:

In [9]:
def get_holiday(year):

    url = f'https://www.timeanddate.com/holidays/estonia/{year}?hol=1' # entramos en la pagina web segun el año

    response = requests.get(url)
    bool(response)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    x = soup.find('section', attrs={'class': 'table-data__table'})
    
    fechas = [] # creamos lista donde almacenaremos las fechas

    for f in x.find_all('tr', attrs={'class': 'showrow'}):
        fecha = f.find('th', attrs={'class':'nw'}).text # obtenemos el mes y dia de la pagina web
        fecha = fecha + ' ' + str(year) # añadimos el año a la fecha
        fecha = replace_month(fecha) # utilizamos función para reemplazar el nombre del mes
        fecha = datetime.strptime(fecha, '%d %b %Y') # convertimos a formato fecha
        fecha = fecha.strftime('%Y-%m-%d') # modificamos al formato fecha ingles
        fechas.append(fecha) # lo agregamos a la lista
        
    return fechas

En este codigo accedemos a una web que nos indica la fecha de los festivos en Estonia, lo cual obtenemos mediante web-scrapping, el problema vino a raiz de que la fecha nos la aportaba en formato español, el cual suponia un problema a la hora de convertirlo a formato fecha ya que no lo identificaba y es por esto que creamos la función "replace_month" la cual nos permite modificar los datos para poder convertirlo a formato fecha

<br>
<br>

### Función de reemplazo de meses

La función es:

In [10]:
def replace_month(spanish_date):
    months_translation = {
        'ene': 'Jan',
        'feb': 'Feb',
        'mar': 'Mar',
        'abr': 'Apr',
        'may': 'May',
        'jun': 'Jun',
        'jul': 'Jul',
        'ago': 'Aug',
        'sep': 'Sep',
        'oct': 'Oct',
        'nov': 'Nov',
        'dic': 'Dec'
    }

    # Reemplazar los nombres de los meses en español por sus equivalentes en inglés
    for month_es, month_en in months_translation.items():
        spanish_date = spanish_date.replace(f'de {month_es}', month_en)

    return spanish_date

Con esta función conseguimos modificar esos datos para que pueda interpretarlo como fecha

<br>
<br>

### Funcion transformación de "df"

La función es:

In [11]:
def df_transformation_total(df):
    
    # Lagged_target
    df['lagged_target_by_weather_1'] = df.groupby(['prediction_unit_id', 'is_consumption', 'hour', 'labels'])['target'].shift(1)
    df['lagged_target_by_weather_2'] = df.groupby(['prediction_unit_id', 'is_consumption', 'hour', 'labels'])['target'].shift(2) 
    df['lagged_target_by_weather_3'] = df.groupby(['prediction_unit_id', 'is_consumption', 'hour', 'labels'])['target'].shift(3)
    
    df['lagged_target_specific_weather'] = df.groupby(['prediction_unit_id', 'is_consumption', 'hour', 'specific_labels'])['target'].shift(1)
    
    # Creamos una nueva variable
    df['rad_install_cap_relation'] = df['shortwave_radiation'] / df['installed_capacity']
    
    # Eliminamos las filas en las que hay nulos
    df = df.dropna()
    
    # Eliminamos las columnas que no aportan valor
    columnas_a_eliminar = ['datetime', 'data_block_id', 'prediction_unit_id', 'date']
    df = df.drop(columnas_a_eliminar, axis=1)
    
    # Modificamos el tipo de datos de alguna de las variables
    df['county'] = df['county'].astype('category')
    df['product_type'] = df['product_type'].astype('category')
    df['labels'] = df['labels'].astype('category')
    df['specific_labels'] = df['specific_labels'].astype('category')
    df['is_business'] = df['is_business'].astype(bool)
    df['is_consumption'] = df['is_consumption'].astype(bool)
    df['holiday'] = df['holiday'].astype(bool)
    
    return df

Como ya hemos comentado antes queriamos obtener el "target" anterior de clientes con las mismas caracteristicas y diferenciando entre producir y consumir, pero ahora queriamos además que fuese la misma hora y que tuviera las mismas condiciones climáticas y ahora que ya habiamos unido las tablas podiamos hacerlo, añadiendo a la agrupación que hicimos en "train_transformation" las columnas: "hour" y "labels"

En este caso laggeamos 3 veces, mientras que cuando agrupamos por "specific_labels" en lugar de "labels" solo laggeamos una vez, ya que generaba muchos nulos

Aprovechando la union de las diferentes tablas creamos otra variable que relacionaba la radiación de ondas cortas con la capacidad instalada

Posteriormente eliminamos los nulos y las columnas que no aportan valor

Y para finalizar modificamos el tipo de dato de las siguientes variables:
- "county" (categórica)
- "product_type" (categórica)
- "labels" (categórica)
- "specific_labels" (categórica)
- "is_business" (booleana)
- "is_consumption" (booleana)
- "holiday" (booleana)